## Auto Submission Tool For Deepracer

## Import the required Libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import datetime
from selenium.webdriver import ActionChains

## Configuration
* chrome_driver_path : Give path to chrome driver on your PC
* chrome_user_data : This note book does not handle auto login feature. You'll have to login in manually and give the path to your chrome profile.
* TIME_INTERVAL : Sleep time interval , before which the tool can start resubmitting. For time trial races, it usually takes around 7 minutes for each submission with upto 2-3 offtracks.  
* Define the `race link` in a variable and the `list of models` which you want to submit in succession. Add them both to the submissions list as a list.
* The race times are logged in `logRaceTimes.txt`
* Since the submission uses an infinite while loop, please restart kernal in order to stop the submissions

In [ ]:
#Configuration
chrome_driver_path = "C:\\Users\\Rogue\\Downloads\\Compressed\\chromedriver"
chrome_user_data = "C:\\Users\\Rogue\\AppData\\Local\\Google\\Chrome\\User Data"
TIME_INTERVAL=420
LOG_FILE = "logRaceTimes.txt"

r1="https://console.aws.amazon.com/deepracer/home?region=us-east-1#......."
m1=["model1"]
r2 = r1
m2 = ["model1","model2"]

submissions = []
submissions.append([r1,m1])
submissions.append([r2,m2])

In [ ]:
def log_data_to_file(log_data):
        log_file = open(LOG_FILE, "a")
        log_file.write(log_data)
        log_file.close()
        
def submit_to_race(driver, event, model):
    try:
        actionChains = ActionChains(driver)    
        driver.get(event)
        time.sleep(4)
        
        model_name = driver.find_element_by_id("PLCHLDR_latest_model_name").text
        race_time = driver.find_element_by_id("PLCHLDR_latest_model_time").text
        
        # logs data as model,time
        log_data = model_name+" , "+race_time+"\n"
        log_data_to_file(log_data)
        race_again_button = driver.find_element_by_class_name("awsui-button-variant-primary")
        driver.execute_script("arguments[0].click();", race_again_button)
        
        # Wait till element is found and click the dropdown
        wait = WebDriverWait(driver, 60)
        dropdown = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "awsui-select-keyboard-area")))
        actionChains.move_to_element(dropdown).perform()
        dropdown.click()
        time.sleep(1)
    
        # Select the model
        selected_model = driver.find_element_by_xpath("//span[@class='awsui-select-option-label' and text()='"+model+"']")
        selected_model.click()
        
        #Click Submit model
        submit_button = driver.find_element_by_class_name("awsui-button-variant-primary")
        driver.execute_script("arguments[0].click();", submit_button)
        time.sleep(1)         
    except Exception as e :
        print("Unable to submit to the race for the model - ",model,e)
        return

In [ ]:
# Initializing selenium
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--user-data-dir="+chrome_user_data)
driver = webdriver.Chrome(executable_path=chrome_driver_path, chrome_options=options)
driver.maximize_window()

# Submit to Race
itr=0
while True:
    log_data_to_file("Running Iteration : "+ str(itr)+"\n")
    print("Running Iteration : ",itr)
    for entry in submissions : 
        driver.get("https://google.co.in")
        race_link = entry[0]
        model_list = entry[1]
        submit_to_race(driver, race_link, model_list[itr%len(model_list)])
    itr+=1
    time.sleep(1)   
    print("Time : ",datetime.datetime.now().time(),", sleeping for ",round(TIME_INTERVAL/60,2)," minutes")
    time.sleep(TIME_INTERVAL)